In [1]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import selenium.webdriver.support.expected_conditions as ec
import mysql.connector
import random
import math

def getTimestampAgora():
    return time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())

def getConexao():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="projeto"
    )

def existeSala(idSala):
    db      = getConexao()
    cursor  = db.cursor()
    sql     = 'SELECT 1 FROM sala_virtuals WHERE id = {};'.format(idSala)
    cursor.execute(sql)
    return len(cursor.fetchall()) > 0

def verificaInsereSala(idSala, nomeSala):
    timestamp  = getTimestampAgora()
    if not existeSala(idSala):
        db              = getConexao()
        cursor          = db.cursor()
        sql             = "INSERT INTO sala_virtuals (id, nome, descricao, disciplina_id, created_at, updated_at) VALUES ( %s, %s, %s, %s, %s, %s )"
        idSalaImportada = 1
        values          = (idSala, nomeSala, nomeSala, idSalaImportada, timestamp, timestamp)
        cursor.execute(sql, values)
        db.commit()

def existeUsuarioPessoaAluno(idAluno):
    db      = getConexao()
    cursor  = db.cursor()
    sql     = 'SELECT 1 FROM usuarios WHERE id = {};'.format(idAluno)
    cursor.execute(sql)
    return len(cursor.fetchall()) > 0

def insereUsuarioAluno(idAluno):
    timestamp   = getTimestampAgora()
    db          = getConexao()
    cursor      = db.cursor()
    sql         = "INSERT INTO usuarios (id, usuario, senha, ativo, termo, tipo_usuario_id, created_at, updated_at) VALUES ( %s, %s, %s, %s, %s, %s, %s, %s )"
    values      = (idAluno, idAluno, 'aluno{}'.format(math.ceil(random.uniform(0, 10000))), 1, 1, 3, timestamp, timestamp)
    cursor.execute(sql, values)
    db.commit()

def inserePessoaAluno(idAluno, nomeAluno):
    timestamp   = getTimestampAgora()
    db          = getConexao()
    cursor      = db.cursor()
    sql         = "INSERT INTO pessoas (id, nome, data_nascimento, usuario_id, created_at, updated_at) VALUES ( %s, %s, %s, %s, %s, %s )"
    values      = (idAluno, nomeAluno, '2000-01-01', idAluno, timestamp, timestamp)
    cursor.execute(sql, values)
    db.commit()

def insereAluno(idAluno):
    timestamp   = getTimestampAgora()
    db          = getConexao()
    cursor      = db.cursor()
    sql         = "INSERT INTO alunos (pessoa_id, created_at, updated_at) VALUES (%s, %s, %s)"
    values      = (idAluno, timestamp, timestamp)
    cursor.execute(sql, values)
    db.commit()

def verificaInsereAluno(idAluno, nomeAluno):
    if not existeUsuarioPessoaAluno(idAluno):
        insereUsuarioAluno(idAluno)
        inserePessoaAluno(idAluno, nomeAluno)
        insereAluno(idAluno)

def isAlunoVinculadoSala(idAluno, idCurso):
    db      = getConexao()
    cursor  = db.cursor()
    sql     = 'SELECT 1 FROM sala_virtual_alunos WHERE pessoa_id = {} and sala_virtual_id = {};'.format(idAluno, idCurso)
    cursor.execute(sql)
    return len(cursor.fetchall()) > 0

def vinculaAlunoSala(idAluno, idCurso):
    if not isAlunoVinculadoSala(idAluno, idCurso):
        timestamp   = getTimestampAgora()
        db          = getConexao()
        cursor      = db.cursor()
        sql         = "INSERT INTO sala_virtual_alunos (pessoa_id, sala_virtual_id, created_at, updated_at) VALUES ( %s, %s, %s, %s )"
        values      = (idAluno, idCurso, timestamp, timestamp)
        cursor.execute(sql, values)
        db.commit()

#Login no Canvas (X)
browser = webdriver.Chrome()
browser.maximize_window()
browser.get("https://canvas.instructure.com/login/canvas")
browser.find_element(By.CSS_SELECTOR, '#pseudonym_session_unique_id').send_keys('carloshenpincord@gmail.com')
browser.find_element(By.CSS_SELECTOR, '#pseudonym_session_password').send_keys('Carlos1310')
browser.find_element(By.CSS_SELECTOR, '#login_form > div.ic-Login__actions > div.ic-Form-control.ic-Form-control--login > button').click()

#CURSOS NÃO PUBLICADOS ()
cursos = int(browser.find_element(By.CSS_SELECTOR, '#DashboardCard_Container > div > div:nth-child(2) > h2').accessible_name.split('(')[1].split(')')[0])
sleep(5)
for i in range(0, cursos):
    #ACESSANDO DETERMINADO CURSO (X)
    selectorCurso = '#DashboardCard_Container > div > div:nth-child(2) > div > div:nth-child({})'.format(i + 1)
    elementoCurso = browser.find_element(By.CSS_SELECTOR, selectorCurso)
    
    #IMPORTANDO O CURSO (X)
    idCurso   = elementoCurso.find_element(By.TAG_NAME, 'a').get_attribute('href').split('/courses/')[1]
    nomeCurso = elementoCurso.find_element(By.TAG_NAME, 'h3').text
    verificaInsereSala(idCurso, nomeCurso)
    
    #ENTRANDO NO CURSO (X)
    browser.find_element(By.CSS_SELECTOR, '#DashboardCard_Container > div > div:nth-child(2) > div > div:nth-child({}) > div > div.ic-DashboardCard__header_hero'.format(i + 1)).click()

    #PESSOAS DO CURSO (X)
    abaPessoas = 0
    for aba in browser.find_element(By.CSS_SELECTOR, '#section-tabs').find_elements(By.TAG_NAME, 'li'):
        if aba.text == 'Pessoas':
            abaPessoas = aba
            break
    abaPessoas.click()
    sleep(5)
    linhas = browser.find_element(By.CSS_SELECTOR, '#tab-0 > div > div:nth-child(2) > table').find_elements(By.TAG_NAME, 'tr')
    for linha in linhas:
        colunas = linha.text.split('\n')
        if int(len(colunas)) > 3:
            if not colunas[3] == 'Professor' and not colunas[3] == 'Papel':
                idAluno = linha.get_attribute('id').split('_')[1]
                verificaInsereAluno(idAluno, colunas[0])
                vinculaAlunoSala(idAluno, idCurso)         

    #VOLTAR PARA A PAGINA DE CURSOS PARA IMPORTAR O PRÓXIMO CURSO(X)
    browser.find_element(By.CSS_SELECTOR, '#global_nav_dashboard_link').click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#DashboardCard_Container > div > div:nth-child(2) > div > div:nth-child(2)"}
  (Session info: chrome=107.0.5304.107)
Stacktrace:
Backtrace:
	Ordinal0 [0x006B1ED3+2236115]
	Ordinal0 [0x006492F1+1807089]
	Ordinal0 [0x005566FD+812797]
	Ordinal0 [0x005855DF+1005023]
	Ordinal0 [0x005857CB+1005515]
	Ordinal0 [0x005B7632+1209906]
	Ordinal0 [0x005A1AD4+1120980]
	Ordinal0 [0x005B59E2+1202658]
	Ordinal0 [0x005A18A6+1120422]
	Ordinal0 [0x0057A73D+960317]
	Ordinal0 [0x0057B71F+964383]
	GetHandleVerifier [0x0095E7E2+2743074]
	GetHandleVerifier [0x009508D4+2685972]
	GetHandleVerifier [0x00742BAA+532202]
	GetHandleVerifier [0x00741990+527568]
	Ordinal0 [0x0065080C+1837068]
	Ordinal0 [0x00654CD8+1854680]
	Ordinal0 [0x00654DC5+1854917]
	Ordinal0 [0x0065ED64+1895780]
	BaseThreadInitThunk [0x7660FEF9+25]
	RtlGetAppContainerNamedObjectPath [0x77587BBE+286]
	RtlGetAppContainerNamedObjectPath [0x